<a href="https://colab.research.google.com/github/hannz15/COVID19-Monitor/blob/master/Torrent_ke_Google_Drive_with_RAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Torrent Ke Google Drive pake Colab

Klik tombol play (run) di sebelah kiri setiap step kodingannya, tidak perlu run semuanya, sesuaikan dengan kebutuhan

Selamat menikmati


# 1. Install Aplikasi nya
Langsung start aja yak, bonus info nama appnya `transmission`



In [ ]:
#@title <<----<strong>Install paket yang diperlukan</strong>
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
TRANSMISSION = True #@param {type:"boolean"}
RCLONE = True #@param {type:"boolean"}
RAR = True #@param {type:"boolean"}
CLEAN_LEFTOVER_FILES = True #@param {type:"boolean"}
#@markdown Ceklis AUTO_RECONNECT untuk mencegah disconnected!

from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True) 

if TRANSMISSION==True:
  !apt update
  !sudo apt install transmission transmission-cli transmission-daemon
  
  !transmission-daemon --download-dir "/content/"
  !transmission-daemon --peerport "54345"
  !transmission-daemon
  !transmission-remote -si

if RCLONE==True:
  !curl https://rclone.org/install.sh | sudo bash

if RAR==True:
  !wget https://gist.githubusercontent.com/MuhammadSaim/de84d1ca59952cf1efaa8c061aab81a1/raw/ca31cbda01412e85949810d52d03573af281f826/rarreg.key
  !wget https://www.rarlab.com/rar/rarlinux-x64-6.0.2.tar.gz
  !tar -xvf rarlinux-x64-6.0.2.tar.gz
  !mkdir -p /usr/local/bin
  !mkdir -p /usr/local/lib
  !cp /content/rar/rar /usr/local/bin
  !cp /content/rar/unrar /usr/local/bin
  !cp /content/rar/rarfiles.lst /etc
  !cp /content/rar/default.sfx /usr/local/lib
  !cp rarreg.key /etc

if CLEAN_LEFTOVER_FILES==True:
  !rm -r rar sample* MANUAL *.key *.txt *.tar.gz

if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button"); 
      btn.click() 
      }
    
    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect"); 
      btn.click() 
      }
    }
    
  setInterval(ClickConnect,60000)
  '''))

# 2. Pilih sumber torrent
Silahkan pilih sumber torrent dibawah ini

# A. Dari File
Upload file torrent yang pengen di donlot

Bisa di-run berulang-ulang buat nambahin torrent lagi, jadi ga 1 doang donlotnya

In [ ]:
from google.colab import files
 
aplod = files.upload()
 
for fn in aplod.keys():
  print('File "{name}" dengan ukuran {length} bytes udah di upload'.format(
      name=fn, length=len(aplod[fn])))
 
for fn in aplod.keys():
  namaf=fn
 
!transmission-remote -a "/content/{namaf}"

# B. Dari link magnet
Copy paste link magnet-nya atau link download torrent-nya di bawah 

Bisa di-run berulang-ulang buat nambahin torrent lagi, jadi ga 1 doang donlotnya

In [ ]:
#@title Input link magnet Torrent
magnet_link = "" #@param {type:"string"}
#while True:
#    magnet_link = input("Masukkan link magnet atau ketik exit: ")
#    if magnet_link.lower() == "exit":
#        break
!transmission-remote -a "{magnet_link}"

# 3. Status donlot
Udah, tinggal start ntar muncul status download nya, ditinggal, di-stop juga masi jalan donlotannya

Filenya ada di folder "Torrent Download" yak



In [ ]:
!rm *.torrent
import time, os
from IPython.display import clear_output
from subprocess import Popen, PIPE, STDOUT
from IPython.core.magic import register_line_magic
 
@register_line_magic
def ngerun(command):
    process = Popen(command, stdout=PIPE, shell=True, stderr=STDOUT, bufsize=1, close_fds=True)
    for line in iter(process.stdout.readline, b''):
        print(line.rstrip().decode('utf-8'))
    process.stdout.close()
    process.wait()
 
def tampil():
  clear_output()
  %ngerun transmission-remote -l
while True:
  tampil()
  time.sleep(3)

# 4. Last Step
Setelah semua donlotan selesai, run code ini

In [ ]:
!killall transmission-daemon
!killall transmission

Kalo mau restart aplikasi-nya, run code ini

In [ ]:
!transmission-daemon --download-dir "/content/"
!transmission-daemon --peerport "54345"
 
!transmission-daemon
!transmission-remote -si

# 5. Archiving
Kalo mau pas download file yang banyak lebih efisien, bisa pake ini.

Catatan:

1.   Arsip Solid, membuat arsip solid. Membutuhkan waktu lama jika level kompresi disetel ke lima.
2.   Enkripsi, menambahkan password.
3.   Enkripsi Nama File, seperti poin kedua namun membutuhkan password saat akan membuka arsip.
4.   Hapus File Setelah Selesai, menghapus file yang diarsipkan.
5.   Kunci, untuk mencegah pengeditan pada arsip.
6.   Split Archive, memecah file arsip menjadi 500MB untuk setiap file asrip.


In [ ]:
Sumber = "test.img" #@param {type:"string"}
Nama_File = "Tes" #@param {type:"string"}
Level_Kompresi = 3 #@param {type:"slider", min:0, max:5, step:1}
Ukuran_Kamus = 32 #@param {type:"slider", min:16, max:1024, step:16}
#@markdown ---

ARSIP_SOLID = False #@param {type:"boolean"}
ENKRIPSI = False #@param {type:"boolean"}
ENKRIPSI_NAMA_FILE = False #@param {type:"boolean"}
HAPUS_FILE_SETELAH_SELESAI = False #@param {type:"boolean"}
KUNCI_ARSIP = False #@param {type:"boolean"}
SPLIT_ARCHIVE = False #@param {type:"boolean"}
TES_ARSIP = False #@param {type:"boolean"}

###############################
args = "a -ma5 -htb "

if TES_ARSIP == True:
  args += "-t "
if HAPUS_FILE_SETELAH_SELESAI == True:
  args += "-df "
if SPLIT_ARCHIVE == True:
  args += "-v512000k "
if ENKRIPSI == True:
  args += "-p"
if ENKRIPSI_NAMA_FILE == True:
  args += "-hp"
if ARSIP_SOLID == True:
  args += "-s "
if KUNCI_ARSIP == True:
  args += "-k "
  
###############################
def runRar():
  !rar $args "-m$Level_Kompresi" "-md$Ukuran_Kamus" "$Nama_File" "$Sumber"

runRar()

# 7. Mounting Cloud Storage

Pake 2 opsi di bawah ya. Kalo pake Drive for Desktop, cuman bisa pake Google Drive. Kalo Rclone, bisa pake cloud storage provider apapun.

####A. Drive for Desktop


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
#@markdown <center><font size="3">Ga support asterisk ya. Kalo mau asterisk pake command <b>CP</b> di cell baru ya</font></center>
Sumber = "" #@param {type:"string"}
Destinasi = "APA YA" #@param{type:"string"}

#################################

def runCp():
  !mkdir /content/drive/My\ Drive/"$Destinasi"
  !chmod ugo+rw /content/drive/My\ Drive/"$Destinasi"
  !cp -v "$Sumber" /content/drive/My\ Drive/"$Destinasi"

runCp()

##### Unmounting

In [ ]:
drive.flush_and_unmount()

#### B. Rclone

Sebelum di mounting, upload dulu file konfigurasi Rclone pake cell di bawah.

Cara bikin file Rclone.config baca <a href="https://onehack.us/t/colab-notebook-to-compress-videos-using-handbrake/104456/9?u=skghd">di sini</a>.

In [ ]:
#@title <-----Upload File Rclone.config
def moveConfig():
  !mv rclone.conf /root/.config/rclone/rclone.conf

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
moveConfig()
print("Moved rclone.conf to /root/.config/rclone/rclone.conf")

In [ ]:
#@markdown <center><h3>Rclone MOUNT / UNMOUNT</h3>Mount the remote as file system on a mountpoint.</center>
import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re
##########################################

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#4f8bd6;margin-top:0px;">
                  Rclone available config...</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))
content = open("/root/.config/rclone/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)
cache_path="/content/temp/rCloneTemp"
def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    !rclone mount $mountNam.value: $mPoint --user-agent 'Mozilla' --buffer-size 256M --transfers 10 --vfs-cache-mode minimal --vfs-read-chunk-size 500M --vfs-cache-max-size 50G --vfs-cache-max-age 0h0m1s --vfs-cache-poll-interval 0m1s --cache-dir '/content/temp/rCloneTemp' --allow-other --daemon 

    if os.path.isdir(mPoint)== True:
      print(f"Mount success! - \t{mPoint}")
    else:
      print(f"Mount failed! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"Unmounted success! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)

ShowAC()

In [ ]:
 
#@markdown <center><img src="https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png" alt="rclone forum" width=150></center>
 
#@markdown ---
source = "" #@param {type:"string"}
destination = "" #@param {type:"string"}
mode = "copy" #@param ["copy", "check"]
dry_run = False #@param {type:"boolean"}
Extra_args = "" #@param {type:"string"}

########################################
args = "--transfers 20 --checkers 20 --stats-one-line --stats=5s -v --tpslimit 95 --tpslimit-burst 40 "
if mode == "check":
  args += "--one-way "
if dry_run == True:
    args +=" --dry-run "
if logs == True:
  args += " --log-file rClone_log.txt "
if Extra_args != "":
  args +=Extra_args
######################################
def runrClone():
  !rclone --user-agent "Mozilla" "$mode" "$source" "$destination" $args

In [ ]:
!md5sum *.rar > MD5.md5
!cp -v *.rar *.md5 "/content/drive/MyDrive/Torrent Downloads"

In [ ]:
drive.flush_and_unmount()

####HANYA JALANKAN CELL INI KALO UDAH YAKIN!!!

In [ ]:
!rm *.rar

# do not touch
buat debugging